# Assignment 01: Multi-class Classification 
In this Assignment, you will train a deep model on the CIFAR10 from the scratch using PyTorch.

### Basic Imports

In [1]:
from sklearn.metrics import f1_score
import os
import time
import os.path as osp

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms
import torchvision

import matplotlib.pyplot as plt
from PIL import Image

### Hyperparameters

In [2]:
# random seed
SEED = 1 
NUM_CLASS = 100

# Training
BATCH_SIZE = 64
NUM_EPOCHS = 60
EVAL_INTERVAL=1
SAVE_DIR = './log'

# Optimizer
LEARNING_RATE = 1e-3
MOMENTUM = 0.9
STEP=5
GAMMA=0.5


### Device

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device="cpu"


### Dataset


In [4]:
transform_cifar100_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

transform_cifar100_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

train_set = torchvision.datasets.CIFAR100(root='../data', train=True,
                                        download=True, transform=transform_cifar100_train)
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

test_set = torchvision.datasets.CIFAR100(root='../data', train=False,
                                       download=True, transform=transform_cifar100_test)
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

class_names = train_set.classes


100%|██████████| 169001437/169001437 [00:49<00:00, 3447349.76it/s]


Extracting ../data/cifar-100-python.tar.gz to ../data
Files already downloaded and verified


### Model

In [5]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.relu2 = nn.ReLU()
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.relu3 = nn.ReLU()
        
        self.fc1 = nn.Linear(64 * 4 * 4, 256)
        self.dropout1 = nn.Dropout(0.5)
        self.relu4 = nn.ReLU()
        
        self.fc2 = nn.Linear(256, 100)

    def forward(self, x):
        x = self.pool(self.relu1(self.bn1(self.conv1(x))))
        x = self.pool(self.relu2(self.bn2(self.conv2(x))))
        x = self.pool(self.relu3(self.bn3(self.conv3(x))))
        
        x = x.view(-1, 64 * 4 * 4)
        
        x = self.relu4(self.dropout1(self.fc1(x)))
        x = self.fc2(x)
        return x


In [6]:
model=ConvNet()
model.to(device)

ConvNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc1): Linear(in_features=1024, out_features=256, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (relu4): ReLU()
  (fc2): Linear(in_features=256, out_features=100, bias=True)
)

### Optimizer

In [7]:
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
from torch.optim import lr_scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP, gamma=GAMMA)

### Task 1: per batch training/testing
---

Please denfine two function named ``train_batch`` and ``test_batch``. These functions are essential for training and evaluating machine learning models using batched data from dataloaders.

**To do**: 
1. Define the loss function i.e [nn.CrossEntropyLoss()](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html).
2. Take the image as the input and generate the output using the pre-defined SimpleNet.
3. Calculate the loss between the output and the corresponding label using the loss function.

In [8]:
def train_batch(model, image, target):
    """
    Perform one training batch iteration.

    Args:
        model (torch.nn.Module): The machine learning model to train.
        image (torch.Tensor): Batch of input data (images).
        target (torch.Tensor): Batch of target labels.

    Returns:
        torch.Tensor: Model output (predictions) for the batch.
        torch.Tensor: Loss value calculated by the defined loss function loss_fn().
    """
    
    ##################### Write your answer here ##################

    output = model(image)
    loss = criterion(output, target)

    ###############################################################

    return output, loss

In [9]:
##################### Write your answer here ##################
# Define the loss function

class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=1.0):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha

    def forward(self, y_pred, y_true):
        """
        y_pred: raw scores (logits) for each class. [batch_size, num_classes]
        y_true: ground truth labels. [batch_size]
        """
        # Convert y_true labels into one-hot encoding
        y_true_onehot = torch.zeros(y_pred.size(), device=y_pred.device).scatter_(1, y_true.unsqueeze(1).long(), 1)
        
        # Calculate softmax over y_pred for calculating probabilities
        probs = F.softmax(y_pred, dim=1)
        
        # Calculate focal loss
        focal_loss = -self.alpha * (y_true_onehot * torch.log(probs) * (1 - probs) ** self.gamma).sum(dim=1).mean()
        
        return focal_loss


#criterion = nn.L1Loss()
criterion = nn.CrossEntropyLoss()
#criterion = FocalLoss(gamma=0.5)
#criterion = FocalLoss(gamma=2.0)

def one_hot_encoding(target, num_classes):
    return F.one_hot(target, num_classes=num_classes).float()


###############################################################

In [10]:

def test_batch(model, image, target):
    """
    Perform one testing batch iteration.

    Args:
        model (torch.nn.Module): The machine learning model to evaluate.
        image (torch.Tensor): Batch of input data (images).
        target (torch.Tensor): Batch of target labels.

    Returns:
        torch.Tensor: Model output (predictions) for the batch.
        torch.Tensor: Loss value calculated for the batch.
    """

    ##################### Write your answer here ##################
   # model.eval()
   # with torch.no_grad():
    output = model(image)
    loss = criterion(output, target)
        
        
    ###############################################################

    return output, loss

### Model Training

In [11]:

# training_loss = []
# training_acc = []
# testing_loss = []
# testing_acc = []
# start_time = time.time()  # 记录开始时间
# all_preds = []
# all_targets = []
# # 日志文件
# log_file = 'training_log.txt'
# for epoch in range(NUM_EPOCHS):
#     model.train()
#     torch.cuda.empty_cache()

#     ##########################
#     ### Training
#     ##########################

#     running_cls_loss = 0.0
#     running_cls_corrects = 0

#     for batch_idx, (image, target) in enumerate(train_dataloader):
#         image = image.to(device)
#         target = target.to(device)
 
#         outputs, loss = train_batch(model, image, target)
     
#         _, preds = torch.max(outputs, 1)

        
#         loss_data = loss.data.item()
#         if np.isnan(loss_data):
#             raise ValueError('loss is nan while training')
#         running_cls_loss += loss.item()
#         running_cls_corrects += torch.sum(preds == target.data)

#         loss.backward()
#         optimizer.step()
#         optimizer.zero_grad()

#     epoch_loss = running_cls_loss / len(train_set)
#     epoch_acc = running_cls_corrects.double() / len(train_set)

#     print(f'Epoch: {epoch+1}/{NUM_EPOCHS} Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
    
#     #########
#     all_preds.extend(preds.cpu().numpy())
#     all_targets.extend(target.cpu().numpy())
#     f1 = f1_score(all_targets, all_preds, average='weighted')
#     print(f"f1 score in {epoch}th epoch is {f1}")

# ############
#     training_loss.append(epoch_loss)
#     training_acc.append(epoch_acc.cpu().detach().numpy())

#     # change learning rate
#     scheduler.step()


#     ##########################
#     ### Testing
#     ##########################
#     # # eval model during training or in the last epoch
    
#     if (epoch + 1) % EVAL_INTERVAL == 0 or (epoch +1) == NUM_EPOCHS:
#         print('Begin test......')
#         model.eval()
    
#         val_loss = 0.0
#         val_corrects = 0

#         for batch_idx, (image, target) in enumerate(test_dataloader):

#             image = image.to(device)
#             target = target.to(device)

#             # test model
#             ##################
#           #  target_one_hot = one_hot_encoding(target, 10)  # 转换成 one-hot 编码
#             outputs, loss = test_batch(model, image, target)
#           #  outputs, loss = test_batch(model, image, target_one_hot)  # 使用 one-hot 编码后的目标
#             #######################
#             _, preds = torch.max(outputs, 1)
            
#             val_loss += loss.item()
#             val_corrects += torch.sum(preds == target.data)

#         val_loss = val_loss / len(test_set)
#         val_acc = val_corrects.double() / len(test_set)
#         print(f'Test Loss: {val_loss:.4f} Acc: {val_acc:.4f}')
#         testing_loss.append(val_loss)
#         testing_acc.append(val_acc.cpu().detach().numpy())
#     with open(log_file, 'a') as f:
#         f.write(f'Epoch {epoch+1}/{NUM_EPOCHS}\n')
#         f.write(f'Train Accuracy: {epoch_acc:.4f}%\n')
#         f.write(f'Test Accuracy: {val_acc:.4f}%\n')
#         f.write('\n')

#         # save the model in last epoch
#         if (epoch +1) == NUM_EPOCHS:
            
#             state = {
#             'state_dict': model.state_dict(),
#             'acc': epoch_acc,
#             'epoch': (epoch+1),
#             }

#             # check the dir
#             if not os.path.exists(SAVE_DIR):
#                 os.makedirs(SAVE_DIR)

#             # save the state
#             torch.save(state, osp.join(SAVE_DIR, 'checkpoint_%s.pth' % (str(epoch+1))))
            
# end_time = time.time()  # 记录结束时间
# duration = end_time - start_time  # 计算训练时间
# print(f"训练模型用时：{duration}秒")
 

### Task 2: Instance inference
---
The task is to visualizes an image along with model prediction and class probabilities.

**To do**: 
1. Calculate the prediction and the probabilities for each class.
         

In [12]:

# inputs, classes = next(iter(test_dataloader))
# input = inputs[0]
# ##################### Write your answer here ##################
# # input: image, model
# # outputs: predict_label, probabilities
# # predict_label is the index (or label) of the class with the highest probability from the probabilities.
# ###############################################################
# input_tensor = input.unsqueeze(0).to(device)
# model.eval()
# with torch.no_grad():
#     output = model(input_tensor)
#     probabilities = F.softmax(output, dim=1)
#     predict_label = torch.argmax(probabilities)
# # predict_label = 
# predicted_class = class_names[predict_label.item()]
# predicted_probability = probabilities[0][predict_label.item()]
# image = input.numpy().transpose((1, 2, 0))
# plt.imshow(image)
# plt.text(17, 30, f'Predicted Class: {predicted_class}\nProbability: {predicted_probability:.2f}', 
#             color='white', backgroundcolor='black', fontsize=8)
# plt.show()

# # Print probabilities for each class
# print('Print probabilities for each class:')
# for i in range(len(class_names)):
#     print(f'{class_names[i]}: {probabilities[0][i].item():.4f}')

In [13]:
# !pip install torchcp
# from torchcp.classification.scores import THR, APS, SAPS, RAPS
# from torchcp.classification.predictors import SplitPredictor, ClusterPredictor, ClassWisePredictor


# from torch.utils.data import DataLoader
# import torch
# import torch.nn as nn







# log_name="CIFAR100"
# model=torch.load("CIFAR100model")
# results = []
# alphas = [0.05, 0.1, 0.15, 0.2]
# saps_weights = [0.5, 1, 1.5, 2]
# raps_penalties = [0.05, 0.1, 0.15, 0.2]
# raps_kregs = [0, 1, 2, 5]




# alpha=0.1
# weight=1
# penalty=0.1
# kreg=0
# results = []
# all_predict_sets_list=[]
# for alpha in alphas:
# #     for weight in saps_weights:
# #         for penalty in raps_penalties:
# #             for kreg in raps_kregs:
#                 score_functions = [THR(), APS(), SAPS(weight=weight), RAPS(penalty=penalty, kreg=kreg)]
#                 predictors = [SplitPredictor, ClusterPredictor, ClassWisePredictor]

#                 for score_function in score_functions:
#                     for Predictor in predictors:
#                         predictor = Predictor(score_function=score_function, model=model)
#                         predictor.calibrate(train_dataloader, alpha=alpha)

#                         all_predict_sets = []
#                         for batch in test_dataloader:
#                             images, _ = batch
#                             images = images.to(device)
#                             predict_set = predictor.predict(images)
#                             all_predict_sets.extend(predict_set)
#                         all_predict_sets_list.append(all_predict_sets)
#                         result_dict = predictor.evaluate(test_dataloader)

#                         result_str = (f"Dataset: {type(train_set).__name__}, "
#                                       f"Score Function: {score_function.__class__.__name__}, "
#                                       f"Predictor: {Predictor.__name__}, "
#                                       f"Alpha: {alpha}, Weight: {weight if isinstance(score_function, SAPS) else 'N/A'}, "
#                                       f"Penalty: {penalty if isinstance(score_function, RAPS) else 'N/A'}, "
#                                       f"kreg: {kreg if isinstance(score_function, RAPS) else 'N/A'}, "
#                                       f"Coverage Rate: {result_dict['Coverage_rate']}, "
#                                       f"Average Size: {result_dict['Average_size']}")
#                         results.append(result_str)

# # 打印或处理存储的结果
# with open(f'12.27result_{log_name}.txt', 'a') as file:
#     file.write("alpha loop"+"\n")
# for result in results:
#     #print(result)
#     with open(f'12.27result_{log_name}.txt', 'a') as file:
#         file.write(result+"\n")
# for all_predict_set in all_predict_sets_list:
#     #print(all_predict_set)
#     with open(f'12.27result_{log_name}.txt', 'a') as file:
#         file.write(' '.join(all_predict_set)+"\n")
    
    



# alpha=0.1
# weight=1
# penalty=0.1
# kreg=0
# results = []
# all_predict_sets_list=[]
# # for alpha in alphas:
# #     for weight in saps_weights:
# #         for penalty in raps_penalties:
# for kreg in raps_kregs:
#                 score_functions = [THR(), APS(), SAPS(weight=weight), RAPS(penalty=penalty, kreg=kreg)]
#                 predictors = [SplitPredictor, ClusterPredictor, ClassWisePredictor]

#                 for score_function in score_functions:
#                     for Predictor in predictors:
#                         predictor = Predictor(score_function=score_function, model=model)
#                         predictor.calibrate(train_dataloader, alpha=alpha)

#                         all_predict_sets = []
#                         for batch in test_dataloader:
#                             images, _ = batch
#                             images = images.to(device)
#                             predict_set = predictor.predict(images)
#                             all_predict_sets.extend(predict_set)
#                         all_predict_sets_list.append(all_predict_sets)
#                         result_dict = predictor.evaluate(test_dataloader)

#                         result_str = (f"Dataset: {type(train_set).__name__}, "
#                                       f"Score Function: {score_function.__class__.__name__}, "
#                                       f"Predictor: {Predictor.__name__}, "
#                                       f"Alpha: {alpha}, Weight: {weight if isinstance(score_function, SAPS) else 'N/A'}, "
#                                       f"Penalty: {penalty if isinstance(score_function, RAPS) else 'N/A'}, "
#                                       f"kreg: {kreg if isinstance(score_function, RAPS) else 'N/A'}, "
#                                       f"Coverage Rate: {result_dict['Coverage_rate']}, "
#                                       f"Average Size: {result_dict['Average_size']}")
#                         results.append(result_str)

# # 打印或处理存储的结果
# with open(f'12.27result_{log_name}.txt', 'a') as file:
#     file.write("kreg loop"+"\n")
# for result in results:
#     #print(result)
#     with open(f'12.27result_{log_name}.txt', 'a') as file:
#         file.write(result+"\n")
# for all_predict_set in all_predict_sets_list:
#     #print(all_predict_set)
#     with open(f'12.27result_{log_name}.txt', 'a') as file:
#         file.write(' '.join(all_predict_set)+"\n")


# alpha=0.1
# weight=1
# penalty=0.1
# kreg=0
# results = []
# all_predict_sets_list=[]
# # for alpha in alphas:
# for weight in saps_weights:
# #         for penalty in raps_penalties:
# #             for kreg in raps_kregs:
#                 score_functions = [THR(), APS(), SAPS(weight=weight), RAPS(penalty=penalty, kreg=kreg)]
#                 predictors = [SplitPredictor, ClusterPredictor, ClassWisePredictor]

#                 for score_function in score_functions:
#                     for Predictor in predictors:
#                         predictor = Predictor(score_function=score_function, model=model)
#                         predictor.calibrate(train_dataloader, alpha=alpha)

#                         all_predict_sets = []
#                         for batch in test_dataloader:
#                             images, _ = batch
#                             images = images.to(device)
#                             predict_set = predictor.predict(images)
#                             all_predict_sets.extend(predict_set)
#                         all_predict_sets_list.append(all_predict_sets)
#                         result_dict = predictor.evaluate(test_dataloader)

#                         result_str = (f"Dataset: {type(train_set).__name__}, "
#                                       f"Score Function: {score_function.__class__.__name__}, "
#                                       f"Predictor: {Predictor.__name__}, "
#                                       f"Alpha: {alpha}, Weight: {weight if isinstance(score_function, SAPS) else 'N/A'}, "
#                                       f"Penalty: {penalty if isinstance(score_function, RAPS) else 'N/A'}, "
#                                       f"kreg: {kreg if isinstance(score_function, RAPS) else 'N/A'}, "
#                                       f"Coverage Rate: {result_dict['Coverage_rate']}, "
#                                       f"Average Size: {result_dict['Average_size']}")
#                         results.append(result_str)

# # 打印或处理存储的结果
# with open(f'12.27result_{log_name}.txt', 'a') as file:
#     file.write("weight loop"+"\n")
# for result in results:
#     #print(result)
#     with open(f'12.27result_{log_name}.txt', 'a') as file:
#         file.write(result+"\n")
# for all_predict_set in all_predict_sets_list:
#     #print(all_predict_set)
#     with open(f'12.27result_{log_name}.txt', 'a') as file:
#         file.write(' '.join(all_predict_set)+"\n")
        
        

# alpha=0.1
# weight=1
# penalty=0.1
# kreg=0
# results = []
# all_predict_sets_list=[]
# # for alpha in alphas:
# #     for weight in saps_weights:
# for penalty in raps_penalties:
# #             for kreg in raps_kregs:
#                 score_functions = [THR(), APS(), SAPS(weight=weight), RAPS(penalty=penalty, kreg=kreg)]
#                 predictors = [SplitPredictor, ClusterPredictor, ClassWisePredictor]

#                 for score_function in score_functions:
#                     for Predictor in predictors:
#                         predictor = Predictor(score_function=score_function, model=model)
#                         predictor.calibrate(train_dataloader, alpha=alpha)

#                         all_predict_sets = []
#                         for batch in test_dataloader:
#                             images, _ = batch
#                             images = images.to(device)
#                             predict_set = predictor.predict(images)
#                             all_predict_sets.extend(predict_set)
#                         all_predict_sets_list.append(all_predict_sets)
#                         result_dict = predictor.evaluate(test_dataloader)

#                         result_str = (f"Dataset: {type(train_set).__name__}, "
#                                       f"Score Function: {score_function.__class__.__name__}, "
#                                       f"Predictor: {Predictor.__name__}, "
#                                       f"Alpha: {alpha}, Weight: {weight if isinstance(score_function, SAPS) else 'N/A'}, "
#                                       f"Penalty: {penalty if isinstance(score_function, RAPS) else 'N/A'}, "
#                                       f"kreg: {kreg if isinstance(score_function, RAPS) else 'N/A'}, "
#                                       f"Coverage Rate: {result_dict['Coverage_rate']}, "
#                                       f"Average Size: {result_dict['Average_size']}")
#                         results.append(result_str)

# # 打印或处理存储的结果
# with open(f'12.27result_{log_name}.txt', 'a') as file:
#     file.write("penalty loop"+"\n")
# for result in results:
#     #print(result)
#     with open(f'12.27result_{log_name}.txt', 'a') as file:
#         file.write(result+"\n")
# for all_predict_set in all_predict_sets_list:
#     #print(all_predict_set)
#     with open(f'12.27result_{log_name}.txt', 'a') as file:
#         file.write(' '.join(all_predict_set)+"\n")

In [14]:
!pip install torchcp
from torchcp.classification.scores import THR, APS, SAPS, RAPS
from torchcp.classification.predictors import SplitPredictor, ClusterPredictor, ClassWisePredictor


from torch.utils.data import DataLoader
import torch
import torch.nn as nn

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [15]:
def evaluate_and_log_results(model, alphas, weights, penalties, kregs, train_dataloader, test_dataloader, device, log_name):
    results = []
    all_predict_sets_list = []

    for alpha in alphas:
        for weight in weights:
            for penalty in penalties:
                for kreg in kregs:
                    score_functions = [THR(), APS(), SAPS(weight=weight), RAPS(penalty=penalty, kreg=kreg)]
                    predictors = [SplitPredictor, ClusterPredictor, ClassWisePredictor]

                    for score_function in score_functions:
                        for Predictor in predictors:
                            predictor = Predictor(score_function=score_function, model=model)
                            predictor.calibrate(train_dataloader, alpha=alpha)

                            all_predict_sets = []
                            for batch in test_dataloader:
                                images, _ = batch
                                images = images.to(device)
                                predict_set = predictor.predict(images)
                                all_predict_sets.extend(predict_set)
                            all_predict_sets_list.append(all_predict_sets)
                            result_dict = predictor.evaluate(test_dataloader)

                            result_str = (f"Dataset: {type(train_set).__name__}, "
                                          f"Score Function: {score_function.__class__.__name__}, "
                                          f"Predictor: {Predictor.__name__}, "
                                          f"Alpha: {alpha}, Weight: {weight if isinstance(score_function, SAPS) else 'N/A'}, "
                                          f"Penalty: {penalty if isinstance(score_function, RAPS) else 'N/A'}, "
                                          f"kreg: {kreg if isinstance(score_function, RAPS) else 'N/A'}, "
                                          f"Coverage Rate: {result_dict['Coverage_rate']}, "
                                          f"Average Size: {result_dict['Average_size']}")
                            results.append(result_str)

    # Write results to file
    with open(f'{log_name}_results_new.txt', 'a') as file:
        for result in results:
            file.write("***************")
            file.write(result + "\n")
        # for all_predict_set in all_predict_sets_list:
        #     predict_set_str = [' '.join(map(str, sublist)) for sublist in all_predict_set]
        #     file.write('\n'.join(predict_set_str) + "\n")
            


In [ ]:
log_name="CIFAR100"
model=torch.load("CIFAR100model")
alphas = [0.05, 0.1, 0.15, 0.2]
saps_weights = [0.5, 1, 1.5, 2]
raps_penalties = [0.05, 0.1, 0.15, 0.2]
raps_kregs = [0, 1, 2, 5]

evaluate_and_log_results(model, [0.1], [1], [0.1], raps_kregs, train_dataloader, test_dataloader, device, log_name)
evaluate_and_log_results(model, [0.1], [1], raps_penalties, [0], train_dataloader, test_dataloader, device, log_name)
evaluate_and_log_results(model, [0.1], saps_weights, [0.1], [0], train_dataloader, test_dataloader, device, log_name)
evaluate_and_log_results(model, alphas, [1], [0.1], [0], train_dataloader, test_dataloader, device, log_name)

In [ ]:
#torch.save(model,"CIFAR100model")

In [ ]:
# for i in np.array(training_acc):
#     print(f"{i:.5f}",end=',')
# print("\n\n")
# for i in np.array(testing_acc):
#     print(f"{i:.5f}",end=',')

In [ ]:
# fig, ax = plt.subplots()
# epochs = list(range(1, 301))
# line1 = ax.plot(epochs, np.array(training_acc), label="train acc")
# line2 = ax.plot(epochs, np.array(testing_acc), label="test acc")
# ax.set_xlabel('epochs')
# ax.set_ylabel('accuracy')
# ax.grid(True)
# ax.set_ylim(0, 1)
# ax.legend()
# plt.show()